### Model Components

#### Sets
- $U$: Set of exam subjects.
- $R$: Set of classrooms.
- $T$: Set of time slots.

#### Parameters
- $a_u$: Number of students for subject $u$.
- $c_r$: Capacity of classroom $r$.
- $d_t$: Equals 2 if time slot $t$ is either Thursday or Friday afternoon; otherwise, equals 1.

#### Variables
- $x_{u,r,t}$: Binary decision variable, equals 1 if subject $u$ is scheduled in classroom $r$ during time slot $t$, otherwise 0.

### Model

**Objective Function:**
$$
\text{maximize} \quad \sum_{u \in U} \sum_{r \in R} \sum_{t \in T} x_{u,r,t}
$$
The goal is to maximize the number of scheduled exams.

**Constraints:**

1. **Each subject can be scheduled at most once:**
   $$
   \sum_{r \in R} \sum_{t \in T} x_{u,r,t} \leq 1 \quad \forall u \in U
   $$

2. **Classroom capacity constraints (prevent scheduling if student numbers exceed classroom capacity):**
   $$
   x_{u,r,t} = 0 \quad \text{if } a_u > c_r \quad \forall u \in U, \forall r \in R, \forall t \in T
   $$

3. **Each classroom can have at most one exam per time slot:**
   $$
   \sum_{u \in U} x_{u,r,t} \leq 1 \quad \forall r \in R, \forall t \in T
   $$

4. **Desirable time slots (Thursday and Friday afternoons) can have at most two exams across all rooms, while less desirable time slots (weekends) can have at most one:**
   $$
   \sum_{r \in R} \sum_{u in U} x_{u,r,t} \leq d_t \quad \forall t \in T
   $$

### Model Solution
The model uses the above linear constraints and objective function solved via the Gurobi optimizer to determine the optimal exam scheduling strategy.


In [1]:
import gurobipy as gp
from gurobipy import GRB

# 创建模型
m = gp.Model("exam_scheduling")

# 单元数据
units = {
    "QBUS1040": 250, "QBUS2310": 150, "QBUS2810": 175, 
    "QBUS2820": 125, "QBUS3310": 75, "QBUS3320": 25, 
    "QBUS3820": 50, "QBUS3850": 25
}

# 教室数据
rooms = {
    "1110": 300, "2140": 100
}

# 时间段数据
timeslots = ["Thursday 2-5pm", "Friday 2-5pm", "Saturday 10am-1pm", "Sunday 10am-1pm"]

# 创建变量
x = m.addVars(units.keys(), rooms.keys(), timeslots, vtype=GRB.BINARY, name="x")

# 确保每个单元最多只能被安排一次
for unit in units:
    m.addConstr(x.sum(unit, '*', '*') <= 1, name=f"assign_once_{unit}")

# 教室容量约束
for unit in units:
    for room in rooms:
        for timeslot in timeslots:
            if units[unit] > rooms[room]:
                m.addConstr(x[unit, room, timeslot] == 0, name=f"cap_{unit}_{room}_{timeslot}")

# 特定时间段的考试安排限制
# 周四和周五下午每个时间段每个教室最多可安排一场考试，两个教室合计最多两场
for timeslot in ["Thursday 2-5pm", "Friday 2-5pm"]:
    m.addConstr(x.sum('*', '1110', timeslot) <= 1, name=f"max_one_exam_1110_{timeslot}")
    m.addConstr(x.sum('*', '2140', timeslot) <= 1, name=f"max_one_exam_2140_{timeslot}")
    m.addConstr(x.sum('*', '*', timeslot) <= 2, name=f"total_max_two_exams_{timeslot}")

# 周末的每个时间段两个教室最多安排一场考试
for timeslot in ["Saturday 10am-1pm", "Sunday 10am-1pm"]:
    m.addConstr(x.sum('*', '*', timeslot) <= 1, name=f"weekend_max_one_exam_{timeslot}")

# 设置目标函数为最大化安排的考试数量
m.setObjective(x.sum(), GRB.MAXIMIZE)

# 求解模型
m.optimize()

# 打印结果
if m.status == GRB.OPTIMAL:
    for unit in units:
        for room in rooms:
            for timeslot in timeslots:
                if x[unit, room, timeslot].X > 0.5:
                    print(f"Unit {unit} is scheduled in room {room} at {timeslot}")


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 32 rows, 64 columns and 176 nonzeros
Model fingerprint: 0xd96ca40b
Variable types: 0 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 6.0000000
Presolve removed 18 rows and 24 columns
Presolve time: 0.00s
Presolved: 14 rows, 40 columns, 80 nonzeros
Variable types: 0 continuous, 40 integer (40 binary)

Root relaxation: cutoff, 17 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     c